# Import Dependencies

In [1]:
import re
import pickle
import string
import numpy as np
import pandas as pd

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.attrs import IS_ALPHA
from spacy.lang.en import English
from spacy import displacy, lemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from wordcloud import WordCloud
from textwrap import wrap

import pyLDAvis
import pyLDAvis.sklearn

# Custom functions from .py files
from web_scrape import get_transcript
from preprocess import clean, preprocess

# Import/Preprocess Data

In [2]:
# Web scrape most recent speeches using our imported 'get_transcript'
goodyear = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-goodyear-az-october-28')
bullhead = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-bullhead-city-az-october-28')
omaha = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-omaha-ne-october-27')
wsalem = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-west-salem-wisconsin-october-27')
lansing = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lansing-michigan-october-27')
martinsburg = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-martinsburg-pa-october-26')
lititz = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lititz-pa-october-26')
allentown = get_transcript('https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-allentown-pa-october-26')

In [3]:
# Clean all of the speeches in the corpus using our imported 'clean' function
clean_goodyear = clean(goodyear)

In [4]:
# Preprocess the speeches using our imported 'preprocess' function
processed_goodyear = preprocess(clean_goodyear)

# Doc-Term Matrix

In [5]:
# Countvectorizer
cv = CountVectorizer(stop_words='english')

X_cv = cv.fit_transform(processed_goodyear)

df_cv = pd.DataFrame(X_cv.toarray(), columns=cv.get_feature_names())

In [6]:
# TF_IDF
tfidf_vectorizer = TfidfVectorizer(**cv.get_params())

X_tfidf = tfidf_vectorizer.fit_transform(processed_goodyear)

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1795: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


# LDA

In [12]:
# for TF DTM
lda_cv = LatentDirichletAllocation(n_components=2, random_state=0)
lda_cv.fit(df_cv)

lda_tfidf = LatentDirichletAllocation(n_components=2, random_state=0)
lda_tfidf.fit(df_tfidf)

LatentDirichletAllocation(n_components=2, random_state=0)

In [13]:
# LDA Visualization for CountVectorizer
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_cv, X_cv, cv)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.119198  0.0       1        1  50.539174
0     -0.119198  0.0       2        1  49.460826, topic_info=        Term       Freq      Total Category  logprob  loglift
382    great  52.000000  52.000000  Default  30.0000  30.0000
1064     win  32.000000  32.000000  Default  29.0000  29.0000
812    right  34.000000  34.000000  Default  28.0000  28.0000
485      job  28.000000  28.000000  Default  27.0000  27.0000
372     good  24.000000  24.000000  Default  26.0000  26.0000
...      ...        ...        ...      ...      ...      ...
557      lot   8.260168  15.245779   Topic2  -5.3516   0.0911
966    thing   8.226208  15.246188   Topic2  -5.3558   0.0870
644   number   7.472771  14.487995   Topic2  -5.4518   0.0419
967    think   7.801271  27.527424   Topic2  -5.4088  -0.5569
545   little   7.073369  12.191026   Topic2  -5.5067   0.1596

[125 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
28        1  0.835910    america
28        2  0.156733    america
29        1  0.708485   american
29        2  0.272494   american
30        1  1.003058  americans
...     ...       ...        ...
1077      2  0.072490       work
1078      1  0.138572      world
1078      2  0.831429      world
1088      1  0.386206       year
1088      2  0.617929       year

[124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [ ]:
# LDA Visualization for TF_IDF
pyLDAvis.sklearn.prepare(lda_tfidf, X_tfidf, tfidf_vectorizer)